In [64]:
!pip install PyPDF2
!pip install pdfplumber
!pip install transformers
!pip install datasets

In [65]:
import spacy
from spacy.matcher import Matcher
import numpy as np
import pandas as pd
import PyPDF2, pdfplumber


In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
nlp = spacy.load("en_core_web_sm")

In [68]:
import os

# Get the folder path
folder_path1 = "/content/drive/MyDrive/data/"

# Get the list of all files and directories in the folder
files1 = os.listdir(folder_path1)
print(files1)

['FITNESS', 'TEACHER', 'HEALTHCARE', 'INFORMATION-TECHNOLOGY', 'FINANCE', 'ENGINEERING', 'PUBLIC-RELATIONS', 'SALES', 'DIGITAL-MEDIA', 'HR', 'AUTOMOBILE', 'BPO', 'BUSINESS-DEVELOPMENT', 'ARTS', 'CHEF', 'DESIGNER', 'BANKING', 'AVIATION', 'CONSTRUCTION', 'CONSULTANT', 'ACCOUNTANT', 'APPAREL', 'AGRICULTURE', 'ADVOCATE']


In [69]:
CV_files = []
pages = []
for i in range(len(files1)):
  files = os.listdir(folder_path1 + files1[i])
  print(files)
  temp = []
  temp1 = []
  for j in range(len(files)):
    a = open(folder_path1 + files1[i] + '/' + files[j],'rb')
    Script=PyPDF2.PdfReader(a)
    temp1.append(len(Script.pages))
    temp.append(a)
  CV_files.append(temp)
  pages.append(temp1)


['10235429.pdf', '19938081.pdf', '14107571.pdf', '17658471.pdf', '19599699.pdf', '11522068.pdf', '12791244.pdf', '19037403.pdf', '10428916.pdf', '10268614.pdf', '12923795.pdf', '17660419.pdf', '10568350.pdf', '13037145.pdf', '10969918.pdf', '16474898.pdf', '17163375.pdf', '16605352.pdf', '11130200.pdf', '12092347.pdf', '20255404.pdf', '19774173.pdf', '13367322.pdf', '12938389.pdf', '18767449.pdf', '17166018.pdf', '15932017.pdf', '17576030.pdf', '12019284.pdf', '17286050.pdf', '11332602.pdf', '14515954.pdf', '17431847.pdf', '12695799.pdf', '15293959.pdf', '16038911.pdf', '10333051.pdf', '19975121.pdf', '17915015.pdf', '13675377.pdf', '10816645.pdf', '28321954.pdf', '20457611.pdf', '27084878.pdf', '29573698.pdf', '20565486.pdf', '22093368.pdf', '24767027.pdf', '30757456.pdf', '24708621.pdf', '24905947.pdf', '23004695.pdf', '29425788.pdf', '26459032.pdf', '23063986.pdf', '28669770.pdf', '29306433.pdf', '24444525.pdf', '25482567.pdf', '29165698.pdf', '24219583.pdf', '28172023.pdf', '242515

In [70]:
Script2 = []
for k in range(len(CV_files)):

  Script1 = []
  for j in range(3):
    Script = []
    with pdfplumber.open(CV_files[k][j]) as pdf:
        for i in range (0,pages[k][j]):
          page=pdf.pages[i]
          text=page.extract_text()
          #print (text)
          Script.append(text)
    Script1.append(Script)
  Script2.append(Script1)

In [71]:
CV_Clear = []
for k in range(len(CV_files)):
  temp = []
  for j in range(3):
    Script2[k][j] = ''.join(Script2[k][j])
    temp.append(Script2[k][j].replace("\n",""))
  CV_Clear.append(temp)


In [72]:
resume = []
for i in range(len(CV_Clear)):
  temp = []
  for j in range(3):


    doc = nlp(CV_Clear[i][j])

    # Extract the skills and experience from the resume
    skills = []
    job_roles = []
    education = []
    experience = []

    for entity in doc.ents:
        if entity.label_ == "SKILL":
            skills.append(entity.text)
        elif entity.label_ == "ORG":
            experience.append(entity.text)
        elif entity.label_ == "JOB_ROLE":
            job_roles.append(entity.text)
        elif entity.label_ == "EDUCATION":
            education.append(entity.text)

    resume1 = skills + job_roles + education + experience
    temp.append(resume1)
  resume.append(temp)

In [73]:
import datasets
dataset = datasets.load_dataset("jacob-hugging-face/job-descriptions", split="train")

In [74]:
job_description = []
for i in range(15):
  job = dataset[i]['job_description']
  job_description.append(job)

In [75]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Load pre-trained DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')



In [76]:
# Tokenize and convert to embeddings
job_token_embeddings = []
for i in range(15):
  job_token = tokenizer.encode(job_description[i], add_special_tokens=True, max_length=512, truncation=True)
  job_embeddings = model(torch.tensor(job_token).unsqueeze(0))[0].mean(dim=1)
  job_token_embeddings.append(job_embeddings)




In [77]:
resume_tokens_embeddings = []
for i in range(len(CV_Clear)):
  temp = []
  for j in range(3):
    if (len(CV_Clear[i][j]) != 0):
      resume_tokens = tokenizer.encode(resume[i][j], add_special_tokens=True,max_length=512, truncation=True)
      resume_embeddings = model(torch.tensor(resume_tokens).unsqueeze(0))[0].mean(dim=1)
      temp.append(resume_embeddings)
  resume_tokens_embeddings.append(temp)


In [78]:
import torch
from torch.nn.functional import cosine_similarity

In [79]:
score = []
for i in range(15):
  temp = []
  for j in range(len(CV_Clear)):
    temp1 = []
    for k in range(3):
      if (len(CV_Clear[j][k]) != 0):
        similarity_score = cosine_similarity(job_token_embeddings[i].unsqueeze(0), resume_tokens_embeddings[j][k].unsqueeze(0), dim=0)
        similarity_score = similarity_score.mean().item()
        temp1.append(similarity_score)
    temp.append(temp1)
  score.append(temp)

In [80]:
print(len(score))

15


In [81]:
top_5 = []
for i in range(14):

  flat_list = [item for sublist in score[i] for item in sublist]

  # Sort the flattened list in descending order
  flat_list.sort(reverse=True)

  # Get the top 5 maximum elements
  top_5_max = flat_list[:5]
  top_5.append(top_5_max)
